# Machine Learning: A Simple Demonstration

There is clearly much talk and buzz around machine learning right now, but what actually is it? Put simply, it is a collection of techniques that aim to uncover patterns in data. It is particularly useful when it is impossible or impractical to define a set of rules. For example a task of recognising damage in cars from photos. It would be impractical to break this down into if...then rules on individual image pixels. Instead we can show a machine learning algorithm a 'labelled' dataset (the training set) where photos have been labelled in some way and the algorithm learns the pattern in order to make a prediction on an unseen photo. This is supervised learning. It can be considered a 'black-box' in the sense that we do not know what rules it has discovered. There are also unsupervised techniques but for simplicity this article uses supervised learning to learn a simple task.

Machine learning is good at generalising relationships within data, but only on data in ranges that it has seen before. It does not discover underlying 'rules' in data (assuming rules even exist). The behaviour of a machine learning model can mimick underlying rules, but it is *not* a 'rules engine'. The following demonstration illustrates what happens when we 'teach' a machine learning algorithm multiplication. 

The Python code used in the examples is provided so the interested reader can explore and experiment. This article is also available as a Jupyter notebook at https://github.com/J-Robinson/ML-examples/blob/master/SVR%20Demo.ipynb.

First import some packages:

In [1]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
import numpy as np

np.set_printoptions(suppress=True)

Let's define some training data. The training data is a set of examples where we know what the output should be. In this example we have a set of training examples where $X_{1} \times X_{2} = Y$. Note that all our examples are integers in the range 0 to 10, so our algorithm will only be trained in this narrow range.

In [2]:
X = [[0, 0], [1, 2], [2, 3], [5, 9], [9, 5], [1, 4], [6, 2], [2, 6], [8, 10], [8, 10], [10, 10]]
y = [    0,      2,      6,     45,     45,      4,     12,     12,      80,     80,      100]


Now train our algorithm. In this case we are using support vector machine regression (SVR), but other algorithms could be used such as gradient boosted trees, random forests, etc. so long as it is non-linear. SVR requires a few parameters to be optimised, so we include a grid search to try all combinations and give use the best parameters.

In [3]:
%%capture
svm = svm.SVR(C=65000, epsilon=0.001)
parameters = [{'kernel': ['rbf'], 
               'gamma': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5],
               'C': [1, 10, 100, 1000, 10000, 50000, 100000, 500000, 1000000]}]
clf = GridSearchCV(svm, parameters, cv=5)
clf.fit(X, y) 

The next step is test some in-sample data. These are examples that were used during the training process.

In [4]:
clf.predict([[2, 3], [5, 9], [6, 2], [10, 10], [8, 10]])

array([ 5.98642676, 44.99222229, 11.99090411, 99.98668557, 80.00675102])

Here we can see how the algorithm has generalised. None of the predictions are integers. It is essentially saying $2 \times 3$ is about $6$. It has not discovered the 'rule' of multiplication. It will never be as accurate as a calculator for this simple task.

We will now see how well our algorithm has learned by asking it to predict multiplication on out-of-sample data, i.e. data it has not seen before, but with the range it was trained.

In [5]:
clf.predict([[1, 8], [6, 6], [4, 6], [4, 5], [9, 8]])

array([ 8.32142354, 36.03135955, 23.96221342, 19.92547805, 72.39190788])

It appears to be roughly as accurate as the in-sample data.

Let's see how well it performs on data outside the range of the training set.

In [6]:
clf.predict([[10,12], [15, 15], [20, 10], [100, 100], [-3, 5]])

array([118.46202251, 210.37052536, 189.20610129,  56.29601154,
       -14.22257141])

Clearly the algorithm cannot perform multiplication reliably on numbers outside the range $0$ to $10$. The further away we are from the range of training data, the worse the prediction. The extreme example of $100 \times 100$ being $\approx56$ is not even vaguely correct. Interestingly it didn't do too badly on $-3 \times 5$ being $\approx-14$ despite not being trained on negative numbers.

The takeaways from this simple example are:
- Machine learning does not output 'rules' that explain its decisions.
- It generalises the relationships within data.
- It cannot be used reliably on data outside the range it has been trained.

There are techniques to improve these limitations such as scaling the inputs (Z-Score, Min-Max, etc.) and transformation of the dependent variable through methods such as log-normal, taking the first order derivative, etc.

### Why would we use machine learning?
Mostly because of it's ability to generalise patterns with data. This example was trivial, but real world problems are not and we do not know what the underlying rules are, or even that they exist. For example, a credit decision can take hundereds or thousands of inputs. The machine learning algorithm can learn the relationship between these inputs and examples of high and low risk. 

It is important to use inputs that are within the range that the algorithm was trained. If we've trained a credit risk algorithm on data only for customers who had lending up to $\$500,000$, it _must_ _not_ be used to predict credit worthiness for customers wanting to take out lending of $\$1,000,000$. When deploying a model into production, this is something that needs to be taken into account.

If a model needs explainability, some algorithms are able to return a 'significant feature' analysis. This is a list of inputs ranked from most important to least important. This tells us what the algorithm considers to be important and can be used as the basis of an explanation on it's decisions. This does have limitations however. Just because a feature is significant does not give us the context of why it is significant. For example an algorithm predicting employment may tell us that a significant factor is whether someone buys a train ticket regularly or not. That is what the data says. It does not tell us why this is significant, just that it is. We would then use this to formulate hypotheses and conduct seperate experiments to verify these hypothesis.

